# Distracted Driver Detection

### SpringBoard Data Science Intensive Course Capstone Project

#### Manasa Raghavan

#### Mentor: Ankit Jain


Statefarm sponsored Kaggle Competition for [Distracted Driver Detection](https://www.kaggle.com/c/state-farm-distracted-driver-detection)





## Objective

**Can computer vision spot distracted drivers?**

The dataset contains images of drivers taken by a dashboard camera doing something in the car - Driving safely, talking on cell phone, texting, reaching behind, talking to passenger, fixing hair or makeup, drinking, operating radio. The goal is to predict the likelihood of what the driver is doing in each image. This is an image classification problem of predicting the class the driver belongs to.

The 2D image captures from the dashboard camera are categorized as

* c0: safe driving
* c1: texting - right
* c2: talking on the phone - right
* c3: texting - left
* c4: talking on the phone - left
* c5: operating the radio
* c6: drinking
* c7: reaching behind
* c8: hair and makeup
* c9: talking to passenger

### Data provided###
**Train Data**
Labelled images of drivers in each of the classes. Contains images from 26 different drivers. drivers_img_list.csv provides the driver ID to image ID mapping

**Test Data**
79K Images of drivers that need to be classified





### Packages Used
The following packages were used in this project:
* Numpy - for formatting
* OpenCV - for image reading and re-sizing
* Scikit Learn - For Test/train split, Kfold, metrics
* Keras - For Convolutional Neural Network Model


In [ ]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix


use_cache = 1
# color type: 1 - grey, 3 - rgb
color_type_global = 1


#### Function to read and re-size images

In [ ]:
def get_im_cv2(path, img_rows, img_cols, color_type=1):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    return resized

#### Function to Get images for each Driver

In [ ]:
def get_driver_data():
    dr = dict()
    path = os.path.join( 'input', 'driver_imgs_list.csv')
    print('Read drivers data')
    f = open(path, 'r')
    line = f.readline()
    while (1):
        line = f.readline()
        if line == '':
            break
        arr = line.strip().split(',')
        dr[arr[2]] = arr[0]
    f.close()
    return dr

#### Function to load Train and Test datasets

In [ ]:
def load_train(img_rows, img_cols, color_type=1):
    X_train = []
    y_train = []
    driver_id = []

    driver_data = get_driver_data()

    print('Read train images')
    for j in range(10):
        print('Load folder c{}'.format(j))
        path = os.path.join( 'input', 'train', 'c' + str(j), '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl, img_rows, img_cols, color_type)
            X_train.append(img)
            y_train.append(j)
            driver_id.append(driver_data[flbase])

    unique_drivers = sorted(list(set(driver_id)))
    print('Unique drivers: {}'.format(len(unique_drivers)))
    print(unique_drivers)
    return X_train, y_train, driver_id, unique_drivers


def load_test(img_rows, img_cols, color_type=1):
    print('Read test images')
    path = os.path.join( 'input', 'test', '*.jpg')
    files = glob.glob(path)
    X_test = []
    X_test_id = []
    total = 0
    thr = math.floor(len(files)/10)
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl, img_rows, img_cols, color_type)
        X_test.append(img)
        X_test_id.append(flbase)
        total += 1
        if total%thr == 0:
            print('Read {} images from {}'.format(total, len(files)))

    return X_test, X_test_id

#### Functions to save / read image data  to/ from cache

In [ ]:
def cache_data(data, path):
    if os.path.isdir(os.path.dirname(path)):
        file = open(path, 'wb')
        pickle.dump(data, file)
        file.close()
    else:
        print('Directory doesnt exists')


def restore_data(path):
    data = dict()
    if os.path.isfile(path):
        file = open(path, 'rb')
        data = pickle.load(file)
    return data


#### Data preprocessing - read normalize train and test datasets

In [ ]:
def read_and_normalize_train_data(img_rows, img_cols, color_type=1):
    cache_path = os.path.join('cache', 'train_r_' + str(img_rows) + '_c_' + str(img_cols) + '_t_' + str(color_type) + '.dat')
    if not os.path.isfile(cache_path) or use_cache == 0:
        train_data, train_target, driver_id, unique_drivers = load_train(img_rows, img_cols, color_type)
        cache_data((train_data, train_target, driver_id, unique_drivers), cache_path)
    else:
        print('Restore train from cache!')
        (train_data, train_target, driver_id, unique_drivers) = restore_data(cache_path)

    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)
    train_data = train_data.reshape(train_data.shape[0], color_type, img_rows, img_cols)
    train_target = np_utils.to_categorical(train_target, 10)
    train_data = train_data.astype('float32')
    train_data /= 255
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, driver_id, unique_drivers


def read_and_normalize_test_data(img_rows, img_cols, color_type=1):
    cache_path = os.path.join('cache', 'test_r_' + str(img_rows) + '_c_' + str(img_cols) + '_t_' + str(color_type) + '.dat')
    if not os.path.isfile(cache_path) or use_cache == 0:
        test_data, test_id = load_test(img_rows, img_cols, color_type)
        cache_data((test_data, test_id), cache_path)
    else:
        print('Restore test from cache!')
        (test_data, test_id) = restore_data(cache_path)

    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.reshape(test_data.shape[0], color_type, img_rows, img_cols)
    test_data = test_data.astype('float32')
    test_data /= 255
    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    return test_data, test_id

#### Function to create image lists based on Driver ID

In [ ]:
def copy_selected_drivers(train_data, train_target, driver_id, driver_list):
    data = []
    target = []
    index = []
    for i in range(len(driver_id)):
        if driver_id[i] in driver_list:
            data.append(train_data[i])
            target.append(train_target[i])
            index.append(i)
    data = np.array(data, dtype=np.float32)
    target = np.array(target, dtype=np.float32)
    index = np.array(index, dtype=np.uint32)
    return data, target, index

#### Function to aggregate CV and Test data over several folds

In [ ]:
def dict_to_list(d):
    ret = []
    for i in d.items():
        ret.append(i[1])
    return ret


def merge_several_folds_mean(data, nfolds):
    a = np.array(data[0])
    for i in range(1, nfolds):
        a += np.array(data[i])
    a /= nfolds
    return a.tolist()

#### Function to create submission file to Kaggle

In [ ]:
def create_submission(predictions, test_id, info):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = info + '_' + str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)

### Core Algorithm of the Model - CNN

In [ ]:
def create_model_14(img_rows, img_cols, color_type=1):
    nb_classes = 10
    # number of convolutional filters in 1st ConvNet layer
    nb_filters = 6
    
    # size of pooling area for max pooling
    nb_pool = 2
    
    # convolution kernel size
    nb_conv = 3 
    
    model = Sequential()
    # Conv Layer1
    model.add(Convolution2D(nb_filters, nb_conv, nb_conv,border_mode='same',                            
                            input_shape=(color_type, img_rows, img_cols)))
    model.add(Activation('relu'))
    

    model.add(Convolution2D(nb_filters, nb_conv, nb_conv, border_mode='same'))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.25))
    
    # Conv Layer2
    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters*2, nb_conv, nb_conv, border_mode='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.3))
    
    # Conv Layer3
    model.add(Convolution2D(nb_filters*4, nb_conv, nb_conv, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters*4, nb_conv, nb_conv, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters*4, nb_conv, nb_conv, border_mode='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.4))
    
    # Conv Layer4
    model.add(Convolution2D(nb_filters*8, nb_conv, nb_conv, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters*8, nb_conv, nb_conv, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters*8, nb_conv, nb_conv, border_mode='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.5))

    # Fully Connected Layer1
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    # Fully Connected Layer2 - Output/ Classifier
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    adm_opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=adm_opt, metrics=["accuracy"])
    
    
    return model

### Main Function to run KFold cross Validation 

In [ ]:
def run_cross_validation(nfolds=10):
    # input image dimensions
    img_rows, img_cols = 72, 104
    batch_size = 128
    nb_epoch = 15
    random_state = 14

    train_data, train_target, driver_id, unique_drivers = read_and_normalize_train_data(img_rows, img_cols, color_type_global)
    test_data, test_id = read_and_normalize_test_data(img_rows, img_cols, color_type_global)

    yfull_train = dict()
    yfull_test = []
    plot_cm = False
    cm_kfold = np.zeros((10,10))
    kf = KFold(len(unique_drivers), n_folds=nfolds, shuffle=True, random_state=random_state)
    num_fold = 0
    for train_drivers, test_drivers in kf:
        unique_list_train = [unique_drivers[i] for i in train_drivers]
        X_train, Y_train, train_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_train)
        unique_list_valid = [unique_drivers[i] for i in test_drivers]
        X_valid, Y_valid, test_index = copy_selected_drivers(train_data, train_target, driver_id, unique_list_valid)

        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))
        print('Train drivers: ', unique_list_train)
        print('Test drivers: ', unique_list_valid)

        model = create_model_14(img_rows, img_cols, color_type_global)
        model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
                  verbose=1, validation_data=(X_valid, Y_valid))


        print len(test_index)
        
        predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
        
        score = log_loss(Y_valid, predictions_valid)
        print('Score log_loss: ', score)
        
        cm = confusion_matrix(Y_valid, predictions_valid, labels =  np.arange(0,10))
        cm_kfold = cm_kfold+cm
        if plot_cm:
            print ('Kfold {}'.format(num_fold))
            print(cm_kfold)
            plt.matshow(cm)
            plt.title('Confusion matrix')
            plt.colorbar()
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            plt.xticks(range(0,10))
            plt.yticks(range(0,10))
            plt.show()
            
        # Store CV predictions
        for i in range(len(test_index)):
            yfull_train[test_index[i]] = predictions_valid[i]

        # Store test predictions
        test_prediction = model.predict(test_data, batch_size=128, verbose=1)
        yfull_test.append(test_prediction)

    score = log_loss(train_target, dict_to_list(yfull_train))
    print('Final log_loss: {}, rows: {} cols: {} nfolds: {} epoch: {}'.format(score, img_rows, img_cols, nfolds, nb_epoch))
    info_string = 'loss_' + str(score) \
                    + '_r_' + str(img_rows) \
                    + '_c_' + str(img_cols) \
                    + '_folds_' + str(nfolds) \
                    + '_ep_' + str(nb_epoch)

    test_res = merge_several_folds_mean(yfull_test, nfolds)
    create_submission(test_res, test_id, info_string)

    if plot_cm:        
        print(cm_kfold)
        plt.matshow(cm_kfold)       
        plt.title('Confusion matrix of fold-'+str(num_fold))
        plt.colorbar()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.xticks(range(0,10))
        plt.yticks(range(0,10))
        plt.show()


In [ ]:
run_cross_validation(6)